# Integrating CrewAI Tools with the Autogen Framework

In this tutorial, we demonstrate how to integrate [CrewAI Tools](https://github.com/crewAIInc/crewAI-tools/tree/main) into the Autogen framework. This process enables smooth interoperability between the two systems, allowing developers to leverage CrewAI's powerful tools within Autogen's flexible agent-based architecture. By the end of this guide, you will understand how to configure agents, convert CrewAI tools for use in Autogen, and validate the integration with a practical example.


## Installation
Install the required packages for integrating CrewAI tools into the Autogen framework.
This ensures all dependencies for both frameworks are installed.

```bash
pip install autogen[interop-crewai]
```

## Imports

Import necessary modules and tools.
- `os` is used to access environment variables.
- `TemporaryDirectory` is used for creating a temporary workspace.
- `Path` helps in handling file paths.
- `FileWriterTool` is the CrewAI tool we will integrate.
- `AssistantAgent` and `UserProxyAgent` are core Autogen classes.
- `CrewAIInteroperability` facilitates the interoperability between Autogen and CrewAI.

In [1]:
import os
from pathlib import Path
from tempfile import TemporaryDirectory

from crewai_tools import FileWriterTool

from autogen import AssistantAgent, UserProxyAgent
from autogen.interoperability.crewai import CrewAIInteroperability

2024-12-13 12:45:47,330 - 8269089792 - __init__.py-__init__:18 - WARNING: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


## Agent Configuration

Configure the agents for the interaction.
- `config_list` defines the LLM configurations, including the model and API key.
- `UserProxyAgent` simulates user inputs without requiring actual human interaction (set to `NEVER`).
- `AssistantAgent` represents the AI agent, configured with the LLM settings.

In [2]:
config_list = [{"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}]
user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
)

chatbot = AssistantAgent(
    name="chatbot",
    llm_config={"config_list": config_list},
)

## Tool Integration

Initialize and register the CrewAI tool with Autogen.
- `crewai_tool` is an instance of the `FileWriterTool` from CrewAI.
- `CrewAIInteroperability` converts the CrewAI tool to make it usable in Autogen.
- `register_for_execution` and `register_for_llm` allow the tool to work with the UserProxyAgent and AssistantAgent.

In [3]:
crewai_tool = FileWriterTool()
crewai_interop = CrewAIInteroperability()
ag2_tool = crewai_interop.convert_tool(crewai_tool)

ag2_tool.register_for_execution(user_proxy)
ag2_tool.register_for_llm(chatbot)

## Demonstration

Demonstrate the integration by writing to a file using the converted CrewAI tool.
- A temporary directory is created to simulate a file operation environment.
- The `message` instructs the chatbot to use the tool to write a specific string into a file.
- `user_proxy.initiate_chat` starts the interaction, with the chatbot processing the request and using the tool.
- Finally, the output file is verified to ensure the integration works correctly.

In [4]:
with TemporaryDirectory() as tmpdirname:
    filename = "tool_result.txt"
    message = f"""Write 'Easy Migration :)' into {filename}.
Use {tmpdirname} dir.
"""
    user_proxy.initiate_chat(recipient=chatbot, message=message, max_turns=2)

    assert Path(tmpdirname, filename).read_text() == "Easy Migration :)"


User (to chatbot):

Write 'Easy Migration :)' into tool_result.txt.
Use /var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/tmpwg6ibnmg dir.


--------------------------------------------------------------------------------
chatbot (to User):

***** Suggested tool call (call_en1Jhr0uYnWnMdQCaZz0WxQq): File_Writer_Tool *****
Arguments: 
{
  "args": {
    "filename": "tool_result.txt",
    "directory": "/var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/tmpwg6ibnmg",
    "overwrite": "yes",
    "content": "Easy Migration :)"
  }
}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION File_Writer_Tool...
Using Tool: File Writer Tool
User (to chatbot):

***** Response from calling tool (call_en1Jhr0uYnWnMdQCaZz0WxQq) *****
Content successfully written to /var/folders/79/kmq3y0pj0rbgrrnf4xlq8q0c0000gn/T/tmpwg6ibnmg/tool_result.txt
****************************